In [8]:
import pandas as pd
import janitor   # for .clean_names()

# ————————————————
# 0) Load & clean raw data
# ————————————————
leads = pd.read_csv(
    'data/all_time_leads.csv',
    dtype=str,       # read everything as string
    low_memory=False
).clean_names()

brizo = pd.read_csv(
    'data/combined.csv',
    dtype=str,
    low_memory=False
).clean_names()

# ————————————————
# 1) Normalize phone_number in brizo, build counts
# ————————————————
brizo['phone_number'] = (
    brizo['phone_number']
      .str.replace(r'\D+', '', regex=True)
      .replace('', pd.NA)
)

brizo_counts = (
    brizo
      .dropna(subset=['brizo_id','phone_number'])
      .groupby(['brizo_id','phone_number'], as_index=False)
      .size()
      .rename(columns={'size':'count'})
)

# ————————————————
# 2) Prepare selected_leads & melt hierarchy
# ————————————————
selected_leads = (
    leads[[
      'lead_id','mobile_primary','business_phone_line_1','brizo_id',
      'created_date','first_mel_timestamp','latest_mql_timestamp',
      'assigned_time_ae_','opportunity_created_date','closed_won_date',
      'stage','unqualified_reason','closed_lost_reason'
    ]]
    .drop_duplicates()
)

# Clean phone columns
for col in ['mobile_primary','business_phone_line_1']:
    selected_leads[col] = (
        selected_leads[col]
          .str.replace(r'\D+', '', regex=True)
          .replace('', pd.NA)
    )

# Null out mobile if it duplicates business
mask = (
    selected_leads['business_phone_line_1'] ==
    selected_leads['mobile_primary']
)
selected_leads.loc[mask, 'mobile_primary'] = pd.NA

# Melt into long form with business first, then mobile
phone_cols = ['business_phone_line_1','mobile_primary']
leads_long = (
    selected_leads
      .melt(
         id_vars=[c for c in selected_leads.columns if c not in phone_cols],
         value_vars=phone_cols,
         var_name='phone_field',
         value_name='phone_number'
      )
      .dropna(subset=['phone_number'])
)

# ————————————————
# 3) Merge phone hierarchy to brizo_counts
# ————————————————
merged = brizo_counts.merge(
    leads_long[['brizo_id','phone_field','phone_number']],
    on=['brizo_id','phone_number'],
    how='left'
)

# Tag match_source
merged['match_source'] = (
    merged['phone_field']
      .map({
         'business_phone_line_1': 'business_phone',
         'mobile_primary':       'mobile_phone'
      })
      .fillna('brizo_id')
)

# Drop helper
merged = merged.drop(columns=['phone_field'])

# ————————————————
# 4) Merge in dates by brizo_id only
# ————————————————
date_cols = [
  'created_date','first_mel_timestamp','latest_mql_timestamp',
  'assigned_time_ae_','opportunity_created_date','closed_won_date'
]
dates = (
    selected_leads
      .drop_duplicates(subset=['brizo_id'])
      .loc[:, ['brizo_id'] + date_cols]
)

final = merged.merge(
    dates,
    on='brizo_id',
    how='left'
)

# 5) Inspect
final.head()
final.shape  # should match brizo_counts.shape[0] rows

(106851, 10)